In [1]:
using ParallelTemperingMonteCarlo
using DelimitedFiles
using BenchmarkTools

In [8]:
using Random,Plots

#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 50000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)



Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [9]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
#plot(temp.t_grid,results.heat_cap)

# data = [results.en_histogram[i] for i in 1:n_traj]
# plot(data)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.0003200864687592529, 0.00033815976954365063, 0.0003445942531901754, 0.0003802707825845997, 0.0004032715230095218, 0.0004199878989840593, 0.0004318847700327343, 0.00044022219194622003, 0.0004990657617204075, 0.0005263184828282834, 0.0005485256528776464, 0.0005842146140545923, 0.0005966832535878485, 0.0006435540165736716, 0.000667087624203487, 0.0009176947821843203, 0.0010093399957202208, 0.0011754568967751382, 0.0012731239485057764, 0.0017817599198514176, 0.0020802513570480023, 0.00202465461362989, 0.0026722771754474437, 0.0031462572611312607, 0.003284862186500759, 0.003270325362290281, 0.003135577810582045, 0.002800286927622311, 0.0024534298269398825, 0.0023025051390735377, 0.0020525873561071917, 0.002001221991868398]
done


In [10]:
using Distributed


global_en_min = results.en_min
global_en_max = results.en_max

delta_en = (global_en_max - global_en_min) / (results.n_bin - 1)

#we need to reset the states. The histograms keep breaking and we get energy nelow what we expect



3.724148398760176e-5

In [11]:
function testingcycles(version::Int64)
    if version == -1
        @sync for i = 1:10000
            @spawn MCRun.ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,1 ,1 ,0 ,0, false, false, i ;delta_en = delta_en)
        end
    elseif version == 0
        @sync for i = 1:10000
            MCRun.ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,1 ,1 ,0 ,0, false, false, i ;delta_en = delta_en)
        end
    elseif version == 1
        Threads.@threads for i = 1:10000
            MCRun.ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,1 ,1 ,0 ,0, false, false, i ;delta_en = delta_en)
        end
    end
end


testingcycles (generic function with 1 method)

In [12]:
mc_params,mc_states,step = restart_ptmc(pot)

time1 = @benchmark testingcycles(-1)

mc_params,mc_states,step = restart_ptmc(pot)
time2 = @benchmark testingcycles(0)

mc_params,mc_states,step = restart_ptmc(pot)
time3 = @benchmark testingcycles(1)


CompositeException: Error: too many moves out of binding sphere
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:35
  [2] atom_displacement(pos::StaticArraysCore.SVector{3, Float64}, max_displacement::Float64, bc::SphericalBC{Float64})
    @ ParallelTemperingMonteCarlo.MCMoves ~/ParallelTemperingMonteCarlo.jl/src/MCMoves.jl:97
  [3] atom_move!(mc_state::MCState{Float64, 13, SphericalBC{Float64}}, i_atom::Int64, pot::ELJPotentialEven{6, Float64}, ensemble::NVT)
    @ ParallelTemperingMonteCarlo.MCRun ~/ParallelTemperingMonteCarlo.jl/src/MCRun.jl:216
  [4] mc_step!(mc_state::MCState{Float64, 13, SphericalBC{Float64}}, pot::ELJPotentialEven{6, Float64}, ensemble::NVT, a::Int64, v::Int64, r::Int64)
    @ ParallelTemperingMonteCarlo.MCRun ~/ParallelTemperingMonteCarlo.jl/src/MCRun.jl:242
  [5] mc_cycle!(mc_states::Vector{Any}, move_strat::MoveStrategy{13, 0, 0}, mc_params::MCParams, pot::ELJPotentialEven{6, Float64}, ensemble::NVT, n_steps::Int64, a::Int64, v::Int64, r::Int64)
    @

In [7]:
println("spawn=",time1," normal= ",time2," threaded =", time3)

UndefVarError: UndefVarError: time2 not defined